## Работа с БД: PostgreSQL

psycopg2 - библиотека для подключения к Postgres. Postgres - реляционная СУБД. Она содержит следующие сущности
* database - база данных
* table - таблица

Таблицы могут соединяться друг с другом с помощью операции **join** - то есть зависят друг от друга, боэтому такой тип хранилищ данных называется *реляционным*

Каждая таблица состоит из *кортежей* (или, по простому, строк). Данные из БД можно читать построчно.

Чтобы читать таблицу из БД, нужно авторизоваться с импользованием логина и пароля.

In [1]:
import psycopg2

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

Выполним простейший запрос, который выведет все таблицы в базе *public*:

In [2]:
sql_str = "SELECT table_name FROM information_schema.tables WHERE table_schema='public';"

cursor.execute(sql_str)
tables_data = [a for a in cursor.fetchall()]
conn.commit()

print("Какие таблицы содержаться в Postgres: %s" % tables_data)

Какие таблицы содержаться в Postgres: [('keywords',), ('links',), ('ratings',)]


Postrgres - мощный аналитический инструмент, которорый позволяет перенести сложные расчёты из python на уровень ниже - в СУБД, которая хранит данные.ый аналитический запрос: посчитаем среднюю оценку по фильмам, у которых более 10 оценок и выведем top-5 таких фильмов с самой максимальной оценкой

In [3]:
import psycopg2

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

# запрос
sql_str = """
    SELECT 
        movieId, AVG(rating) as avg_rating
    FROM public.ratings 
    GROUP BY movieId
    HAVING COUNT(RATING)>10
    ORDER BY avg_rating DESC
    LIMIT 10;
"""
cursor.execute(sql_str)


for row in cursor:
        print('movieId: %s \t avg_rating: %s' % row)
conn.close()
print("\nТипизация строки %s" % type(row))

movieId: 632 	 avg_rating: 4.63636363636364
movieId: 142115 	 avg_rating: 4.625
movieId: 99764 	 avg_rating: 4.58333333333333
movieId: 7096 	 avg_rating: 4.57142857142857
movieId: 4046 	 avg_rating: 4.55555555555556
movieId: 8724 	 avg_rating: 4.5
movieId: 111778 	 avg_rating: 4.5
movieId: 5169 	 avg_rating: 4.5
movieId: 4405 	 avg_rating: 4.5
movieId: 159817 	 avg_rating: 4.47826086956522

Типизация строки <class 'tuple'>
